In [ ]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os


test set: https://forums.fast.ai/t/how-to-add-a-test-set/38956/7
text.data reference methods: https://docs.fast.ai/text.html

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/clillianhong/experiment/master/scripts/data/type_data/type_features_produce.csv")

In [ ]:
df = pd.DataFrame({'label':data.label, 'text':data.text})

In [ ]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

#remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:

detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
df['text'] = detokenized_doc

In [ ]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn_l, df_val_l = train_test_split(df, stratify = df['label'], test_size = 0.1, random_state = 12)
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.3, random_state = 12)

df_trn.shape, df_val.shape

In [ ]:
#language model data - vectorization 
data_lm = TextLMDataBunch.from_df(train_df=df_trn_l, valid_df=df_val_l, path = "") #potentially change to use all of the data 

# Classifier model data - 
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)


In [ ]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.5)


In [ ]:
learn.lr_find(start_lr=1e-5, end_lr=10000)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3, moms=(0.8,0.7)) #decrease learning rate by a factor of 10? why? 

In [ ]:
learn.save_encoder('ft_enc_best')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc_best')
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn1 = learn
learn1.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))
learn1.save('clas_first') 


In [ ]:
learn1.freeze_to(-2)
learn1.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))  #decrease learning rate by a factor of 10? 
learn1.save('clas_first_1')

In [ ]:
learn1.freeze_to(-3)
learn1.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))  #decrease learning rate to middle of spike down? (factor of 10?)
learn1.save('clas_first_2')


In [ ]:
learn1.unfreeze()
learn1.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))  #decrease lr slightly further

In [ ]:
learn1.export('/content/gdrive/My Drive/type_produce_classifier.pkl')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn1)
interp.plot_confusion_matrix()